In [1]:
import pandas as pd

LABEL_TRUE = 'Relevant'
LABEL_FALSE = 'Not Relevant'

In [2]:
df_test = pd.read_csv("../data/test.csv")
df_full = pd.read_csv("../data/full.csv", encoding='latin_1')

print(f'Len of test set: {len(df_test)}')

df_merged = pd.merge(df_test, df_full[['text', 'choose_one']], on='text', how='left')
df_merged_uniques = df_merged.drop_duplicates(subset='id')

Len of test set: 3263


In [3]:
df_merged_uniques.loc[df_merged_uniques['id'] == 75]

,id,keyword,location,text,choose_one
25,75,ablaze,India,Rape victim dies as she sets herself ablaze: A...,Relevant


In [4]:
df_merged_dups = df_merged[df_merged.duplicated(subset='text', keep=False)]
df_merged_dups_diff = df_merged_dups.groupby('text').filter(lambda x: x['choose_one'].nunique() > 1)
df_merged_dups_diff_uniques = df_merged_dups_diff.drop_duplicates(subset=['text', 'id'])
print(f'{len(df_merged_dups_diff_uniques)} instances with uncertain targets.')

display(df_merged_dups_diff_uniques)

22 instances with uncertain targets.


,id,keyword,location,text,choose_one
296,922,bioterrorism,NaN,To fight bioterrorism sir.,Relevant
302,924,bioterrorism,NaN,To fight bioterrorism sir.,Relevant
622,1931,burning%20buildings,"Dublin City, Ireland",@RockBottomRadFM Is one of the challenges on T...,Can't Decide
631,1964,burning%20buildings,San Francisco,? High Skies - Burning Buildings ? http://t.co...,Not Relevant
970,3094,deaths,NaN,Bigamist and his Û÷firstÛª wife are charged ...,Not Relevant
1062,3374,demolition,NaN,General News Û¢åÊ'Demolition of houses on wat...,Not Relevant
1292,4053,displaced,Pedophile hunting ground,.POTUS #StrategicPatience is a strategy for #G...,Relevant
1298,4056,displaced,Pedophile hunting ground,.POTUS #StrategicPatience is a strategy for #G...,Relevant
1401,4371,earthquake,in the Word of God,@GreenLacey GodsLove &amp; #thankU my sister f...,Relevant
1463,4572,emergency%20plan,NaN,Do you have an emergency drinking water plan? ...,Relevant


In [22]:
def change_pred(id, pred):
    df.loc[df['id'] == id, 'choose_one'] = pred

df = df_merged_uniques.copy()

change_pred(922, LABEL_TRUE)
change_pred(924, LABEL_FALSE)

change_pred(1931, LABEL_FALSE) # No Change
change_pred(1964, LABEL_FALSE)  # No Change

change_pred(3094, LABEL_FALSE)  # apparently not true.. 

change_pred(4053, LABEL_TRUE) 
change_pred(4056, LABEL_TRUE)

change_pred(4371, LABEL_TRUE)  # apparently true...
change_pred(4572, LABEL_TRUE)  

change_pred(4837, LABEL_TRUE)  # :(
change_pred(4930, LABEL_TRUE)  # :(
change_pred(4949, LABEL_FALSE)  # yeeee

change_pred(5679, LABEL_TRUE)  # :(

change_pred(8011, LABEL_TRUE)  
change_pred(10232, LABEL_FALSE)    # wooo this is actually an issue



In [23]:
df_target = df[['id']].copy()
df_target['target'] = df['choose_one'].apply(lambda x: 1 if x == LABEL_TRUE else 0)

display(df_target)

df_target.to_csv("../data/submission2.csv", index=False)

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3470,10861,0
3471,10865,1
3473,10868,1
3474,10874,1
